In [2]:
!git clone https://github.com/Sharp-4rth/temporal_belief_analysis.git

Cloning into 'temporal_belief_analysis'...
remote: Enumerating objects: 488, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 488 (delta 54), reused 48 (delta 24), pack-reused 389 (from 1)
Receiving objects: 100% (488/488), 3.13 MiB | 6.20 MiB/s, done.
Resolving deltas: 100% (302/302), done.


In [ ]:
# Need to restart after:
!pip install convokit[llm]
!pip install convokit

In [2]:
import sys
import os

os.chdir('/content/temporal_belief_analysis/notebooks')
print("Changed working directory to:", os.getcwd())

# Absolute path to src directory
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

Changed working directory to: /content/temporal_belief_analysis/notebooks


In [3]:
import time
!pip install gdown
import zipfile
import nltk
from nltk.corpus import stopwords
from convokit import Corpus, download
import convokit
from temporal_belief.core.timeline_building import TimelineBuilder

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [32]:
# Download and unzip with python (Dataloading):
# !gdown "https://drive.google.com/file/d/1N0U_jUJlOYjdaju2FaU8p87uB22YBxJ0/view?usp=sharing" -O "/content/temporal_belief_analysis/pd_corpus_with_stances100000_chronological.zip" --fuzzy
# !gdown "https://drive.google.com/file/d/1DLFY6JLMZqNjwvNRZmhlV4-rnoQP_eyH/view?usp=sharing" -O "/content/temporal_belief_analysis/merged_corpus_checkpoint_5.zip" --fuzzy
# !gdown "https://drive.google.com/file/d/1nWaj5N8nsG7u5homv_kAh4CLPDv01M_Z/view?usp=sharing" -O "/content/temporal_belief_analysis/pd_corpus_with_topics.zip" --fuzzy
# !gdown "https://drive.google.com/file/d/15NMRXEkGRoGjK6TXFBHIMOPjkTyZ0keP/view?usp=sharing" -O "/content/temporal_belief_analysis/pd_corpus_with_stances200000_llm.zip" --fuzzy
# !gdown "https://drive.google.com/file/d/15nVf6Js0KsDxA9HaB0zCXhc8VdcC-Fy-/view?usp=sharing" -O "/content/temporal_belief_analysis/pd_corpus_with_stances75000_llm.zip" --fuzzy
# !gdown "https://drive.google.com/file/d/1dOUvQmtjFrXq0hJvnOsP_ZLqdGEyOVNJ/view?usp=sharing" -O "/content/temporal_belief_analysis/pd_corpus_with_stances100000_llm.zip" --fuzzy
# !gdown "https://drive.google.com/file/d/10HNQHIsFe386oBu_o5s9JOuo-wiaofXC/view?usp=sharing" -O "/content/temporal_belief_analysis/pd_corpus_with_stances150000_llm.zip" --fuzzy
# !gdown "https://drive.google.com/file/d/1hU1evfmh0BUAda8llDzBmXfHsMPGdO87/view?usp=sharing" -O "/content/temporal_belief_analysis/MT-CSD-main.zip" --fuzzy


# zipfile.ZipFile("/content/temporal_belief_analysis/pd_corpus_with_stances100000_chronological.zip").extractall("/content/temporal_belief_analysis")
# zipfile.ZipFile("/content/temporal_belief_analysis/merged_corpus_checkpoint_5.zip").extractall("/content/temporal_belief_analysis")
# zipfile.ZipFile("/content/temporal_belief_analysis/pd_corpus_with_topics.zip").extractall("/content/temporal_belief_analysis")
# zipfile.ZipFile("/content/temporal_belief_analysis/pd_corpus_with_stances200000_llm.zip").extractall("/content/temporal_belief_analysis")
# zipfile.ZipFile("/content/temporal_belief_analysis/pd_corpus_with_stances75000_llm.zip").extractall("/content/temporal_belief_analysis")
# zipfile.ZipFile("/content/temporal_belief_analysis/pd_corpus_with_stances100000_llm.zip").extractall("/content/temporal_belief_analysis")
# zipfile.ZipFile("/content/temporal_belief_analysis/MT-CSD-main.zip").extractall("/content/temporal_belief_analysis")
zipfile.ZipFile("/content/temporal_belief_analysis/data-all-annotations.zip").extractall("/content/temporal_belief_analysis")


In [4]:
corpus = Corpus(filename="/content/temporal_belief_analysis/pd_corpus_with_stances150000_llm_rerun")

In [1]:
from pathlib import Path
import pandas as pd
import re

VALID_RAW = {"favor", "against", "none"}

# if favoring this target => left, else right
FAVOR_IMPLIES_LEFT = {
    "Climate Change is a Real Concern": True,
    "Feminist Movement": True,
    "Hillary Clinton": True,
    "Legalization of Abortion": True,
    "Atheism": True,
}

def normalize_cols(df: pd.DataFrame) -> pd.DataFrame:
    cols = {c: c.strip().lower() for c in df.columns}
    df = df.rename(columns=cols)
    if "tweet" in df.columns: df = df.rename(columns={"tweet": "text"})
    if "opinion towards" in df.columns: df = df.rename(columns={"opinion towards": "opinion_towards"})
    return df

def read_semeval_txt(path):
    """
    Read SemEval .txt with resilient encoding handling.
    Tries UTF-8/UTF-8-SIG/CP1252/LATIN-1 and falls back to a
    replacement-decoded path so parsing never crashes.
    """
    seps = ["\t", None]  # TSV first, else autodetect
    encs = ["utf-8", "utf-8-sig", "cp1252", "latin-1"]

    last_err = None
    for enc in encs:
        for sep in seps:
            try:
                return normalize_cols(
                    pd.read_csv(
                        path,
                        sep=sep,
                        encoding=enc,
                        engine="python",
                        on_bad_lines="skip",   # ignore any malformed rows
                        quoting=3              # QUOTE_NONE; treat quotes literally
                    )
                )
            except UnicodeDecodeError as e:
                last_err = e
                continue
            except Exception as e:
                # if it wasn't an encoding error, try next sep/enc anyway
                last_err = e
                continue

    # Final fallback: decode with replacement to guarantee progress
    with open(path, "r", encoding="cp1252", errors="replace") as f:
        txt = f.read()
    return normalize_cols(
        pd.read_csv(
            StringIO(txt),
            sep="\t",
            engine="python",
            on_bad_lines="skip",
            quoting=3
        )
    )

CLEAN_BAD = re.compile(r"^\s*$|^\.$", re.UNICODE)

def clean_text(s: str) -> str:
    s = (s or "").strip()
    if s.lower() in {"[deleted]", "[removed]"}:
        return ""
    return re.sub(r"\s+", " ", s)

def normalize_raw_stance(v) -> str | None:
    if pd.isna(v): return None
    t = str(v).strip().lower()
    if t == "favour": t = "favor"
    return t if t in VALID_RAW else None

def raw_to_left_right(raw: str | None, target: str | None) -> str | None:
    if raw is None: return None
    if raw == "none": return "neutral"
    if not target: return None
    favor_left = FAVOR_IMPLIES_LEFT.get(target, None)
    if favor_left is None: return None
    if raw == "favor":   return "left-leaning"  if favor_left else "right-leaning"
    if raw == "against": return "right-leaning" if favor_left else "left-leaning"
    return None

def apply_filters(df: pd.DataFrame, split_name: str) -> pd.DataFrame:
    # text clean
    df["text"] = df["text"].astype(str).map(clean_text)
    df = df[~df["text"].map(lambda x: CLEAN_BAD.match(x) is not None)]
    # stance normalize
    df["raw_stance"] = df["stance"].map(normalize_raw_stance)
    df = df[df["raw_stance"].notna()]
    # left/right mapping
    df["stance"] = [raw_to_left_right(r, t) for r, t in zip(df["raw_stance"], df["target"])]
    undecided = df["stance"].isna()
    df.loc[undecided & (df["raw_stance"] == "none"), "stance"] = "neutral"
    df = df[df["stance"].notna()]
    # ids & split
    if "id" in df.columns: df["id"] = df["id"].astype(str)
    else: df["id"] = [f"{split_name}_{i}" for i in range(len(df))]
    df["split"] = split_name
    keep = ["id", "target", "text", "raw_stance", "stance", "split"]
    if "opinion_towards" in df.columns: keep.append("opinion_towards")
    if "sentiment" in df.columns: keep.append("sentiment")
    extras = [c for c in df.columns if c not in keep]
    return df[keep + extras].reset_index(drop=True)

def build_semeval_splits(data_dir, out_dir):
    data_dir = Path(data_dir)
    out_dir = Path(out_dir); out_dir.mkdir(parents=True, exist_ok=True)

    train_df = read_semeval_txt(data_dir / "trainingdata-all-annotations.txt")
    dev_df   = read_semeval_txt(data_dir / "trialdata-all-annotations.txt")

    for name, df in [("train", train_df), ("dev", dev_df)]:
        for needed in ["id", "target", "text", "stance"]:
            if needed not in df.columns:
                raise ValueError(f"{name}: missing column '{needed}'. Got: {list(df.columns)}")

    train_df = apply_filters(train_df, "train").drop_duplicates(subset=["id"])
    dev_df   = apply_filters(dev_df, "dev").drop_duplicates(subset=["id"])

    train_out = out_dir / "semeval16_train.csv"; train_df.to_csv(train_out, index=False)
    dev_out   = out_dir / "semeval16_dev.csv";   dev_df.to_csv(dev_out, index=False)

    test_path = data_dir / "testdata-taskA-all-annotations.txt"
    test_out = None
    if test_path.exists():
        test_df = read_semeval_txt(test_path)
        for needed in ["id", "target", "text", "stance"]:
            if needed not in test_df.columns:
                raise ValueError(f"test: missing column '{needed}'. Got: {list(test_df.columns)}")
        test_df = apply_filters(test_df, "test").drop_duplicates(subset=["id"])
        test_out = out_dir / "semeval16_test.csv"
        test_df.to_csv(test_out, index=False)
        return train_df, dev_df, test_df

    return train_df, dev_df

In [2]:
# --- stance scheme (same as yours) ---
from collections import Counter
STANCE2ID = {"neutral": 0, "left-leaning": 1, "right-leaning": 2}
NORMALIZE = {
    "neutral": "neutral",
    "left": "left-leaning", "left-leaning": "left-leaning",
    "right": "right-leaning", "right-leaning": "right-leaning",
}
VALID = set(STANCE2ID.keys())

BAD_TEXT = {"[removed]", "[deleted]", "."}

def build_arrays_from_semeval(df):
    """
    Replicates your ConvoKit filtering but for SemEval rows.

    Input df columns expected (from our converter):
      id, target, text, raw_stance, stance, split, [opinion_towards], [sentiment]
    Returns:
      utterances, labels, utt_ids, user_ids, topics
    """
    utterances, labels, utt_ids, user_ids, topics = [], [], [], [], []

    # SemEval has no confidence score or main-post threading; treat confidence=1.0
    for _, row in df.iterrows():
        txt = (str(row.get("text") or "")).strip()
        if (not txt) or (txt in BAD_TEXT):
            continue

        raw = row.get("stance")
        lab = NORMALIZE.get(str(raw).strip().lower()) if raw is not None else None
        if lab not in VALID:
            continue

        # In your ConvoKit code you skip when 'stance_error' and stance == 'neutral'.
        # SemEval has no such flag; equivalently, we do nothing here.

        # No per-utterance confidence in SemEval; treat as passing the >= 0.8 threshold.
        # If you want to emulate the threshold literally, you can set conf = 1.0 and check it.

        # Topic: use the SemEval target
        topic = row.get("target")

        # IDs: tweet id is available; fabricate a speaker id like the ConvoKit version
        utt_id = str(row.get("id"))
        user_id = f"tw_{utt_id}"  # one speaker per tweet (no user info in release)

        utterances.append(txt)
        labels.append(lab)
        utt_ids.append(utt_id)
        user_ids.append(user_id)
        topics.append(topic)

    return utterances, labels, utt_ids, user_ids, topics

# --- Example usage with your DataFrames from build_semeval_splits() ---

train_X, train_y, train_utt_ids, train_user_ids, train_topics = build_arrays_from_semeval(train_df)
dev_X,   dev_y,   dev_utt_ids,   dev_user_ids,   dev_topics   = build_arrays_from_semeval(dev_df)

# Optional if you have test:
if 'maybe_test' in locals() and len(maybe_test) == 1:
    test_df = maybe_test[0]
    test_X, test_y, test_utt_ids, test_user_ids, test_topics = build_arrays_from_semeval(test_df)

# Quick sanity checks (should mirror earlier counts)
# from collections import Counter
print(Counter(train_y))  # {'right-leaning': ..., 'neutral': ..., 'left-leaning': ...}

NameError: name 'train_df' is not defined

In [5]:
# === SAME stance scheme as before ===
STANCE2ID = {"neutral":0, "left-leaning":1, "right-leaning":2}
NORMALIZE = {
    "neutral":"neutral",
    "left":"left-leaning", "left-leaning":"left-leaning",
    "right":"right-leaning","right-leaning":"right-leaning",
}
VALID = set(STANCE2ID.keys())
BAD_TEXT = {"[removed]", "[deleted]", "."}

def build_arrays_from_df(df):
    utterances, labels, utt_ids, user_ids, topics = [], [], [], [], []

    for _, row in df.iterrows():
        # --- text filter (same spirit as your ConvoKit filters) ---
        txt = (str(row.get("text") or "")).strip()
        if (not txt) or (txt in BAD_TEXT):
            continue

        # SemEval has no confidence/stance_error; treat as passing confidence >= 0.8
        raw = row.get("stance")  # already 'left-leaning'/'right-leaning'/'neutral' from earlier step
        if not raw:
            continue

        lab = NORMALIZE.get(str(raw).strip().lower())
        if lab not in VALID:
            continue

        topic = row.get("target")            # use SemEval target as topic
        utt_id = str(row.get("id"))          # tweet ID
        user_id = f"tw_{utt_id}"             # no real users in release → synthetic speaker

        utterances.append(txt)
        labels.append(lab)
        utt_ids.append(utt_id)
        user_ids.append(user_id)
        topics.append(topic)

    return utterances, labels, utt_ids, user_ids, topics

# ===== USE IT (pick the split you want) =====
utterances, labels, utt_ids, user_ids, topics = build_arrays_from_df(train_df)

print(len(utterances), len(labels), len(utt_ids), len(user_ids), len(topics))  # should all match
# quick sanity:
from collections import Counter
import pandas as pd

print(Counter(labels))
dfs = [train_df, dev_df] + ([maybe_test[0]] if 'maybe_test' in locals() and maybe_test else [])
all_df = pd.concat(dfs, ignore_index=True)
# utterances, labels, utt_ids, user_ids, topics = build_arrays_from_df(all_df)
# Just train split:
utterances, labels, utt_ids, user_ids, topics = build_arrays_from_df(train_df)

2814 2814 2814 2814 2814
Counter({'right-leaning': 1342, 'neutral': 741, 'left-leaning': 731})


In [4]:
train_df, dev_df, *maybe_test = build_semeval_splits(
    data_dir="/content/temporal_belief_analysis/data-all-annotations",
    out_dir="./semeval_splits"
)

In [6]:
print(train_df["target"].value_counts())
print(train_df["stance"].value_counts())

target
Feminist Movement                   664
Hillary Clinton                     639
Legalization of Abortion            603
Atheism                             513
Climate Change is a Real Concern    395
Name: count, dtype: int64
stance
right-leaning    1342
neutral           741
left-leaning      731
Name: count, dtype: int64


In [ ]:
# === Inputs: For ConvoKit Corpus ===
# from convokit import Corpus
# corpus = Corpus("path_or_url")

STANCE2ID = {"neutral":0, "left-leaning":1, "right-leaning":2}
NORMALIZE = {
    "neutral":"neutral",
    "left":"left-leaning", "left-leaning":"left-leaning",
    "right":"right-leaning","right-leaning":"right-leaning",
}
VALID = set(STANCE2ID.keys())

utterances, labels, utt_ids, user_ids, topics = [], [], [], [], []

for u in corpus.iter_utterances():
    txt = (u.text or "").strip()
    stance = u.meta.get('detected_stance')
    main_post_id = u.conversation_id
    main_post = corpus.get_utterance(main_post_id)
    conv = corpus.get_conversation(main_post_id)

    conf_raw = u.meta.get('stance_confidence')
    # normalize confidence
    try:
        conf = float(conf_raw)
    except (TypeError, ValueError):
        conf = None

    # --------- SKIP problematic utterances ----------
    # missing/empty main post or duplicate of main post
    if not txt or txt in {'[removed]', '[deleted]', '.'}:
        continue

    if not main_post.text or txt == main_post.text:
        continue

    if 'stance_error' in u.meta.keys() and stance == 'neutral':
        continue

    if stance not in {'left-leaning', 'right-leaning', 'neutral'}:
        continue

    # confidence threshold depends on stance
    if stance == 'neutral':
        if conf is None or conf < 0.8:
            continue
    elif stance == 'left-leaning':
        if conf is None or conf < 0.8:
            continue
    elif stance == 'right-leaning':
        if conf is None or conf < 0.8:
            continue

    # --- EDIT HERE: pull topic if you have it, else use None/"" ---

    raw = u.meta.get("detected_stance")

    # ------------------------------------------------

    # Okay also check this from runpod

    if not raw:
        continue
    lab = NORMALIZE.get(str(raw).strip().lower())  # map aliases
    if lab not in VALID:                           # filters Unknown / other
        continue

    # --- EDIT HERE: pull topic if you have it, else use None/"" ---
    topic = u.meta.get("topic")  # or compute from your mapping
    # --------------------------------------------------------------

    utterances.append(txt)
    labels.append(lab)
    utt_ids.append(u.id)
    user_ids.append(u.speaker.id if u.speaker else "unknown")
    topics.append(topic)

In [8]:
# --- Cap max utterances per label ---
from collections import defaultdict
import numpy as np

# Set your per-label maximums here:
MAX_PER_LABEL = {
    "neutral": 50000,
    "left-leaning": 50000,
    "right-leaning": 900,
}

# Shuffle indices first for randomness
all_idx = np.arange(len(labels))
rng = np.random.default_rng(42)
rng.shuffle(all_idx)

# Collect capped indices
label_counts = defaultdict(int)
keep_idx = []

for i in all_idx:
    lab = labels[i]
    if lab in MAX_PER_LABEL and label_counts[lab] < MAX_PER_LABEL[lab]:
        keep_idx.append(i)
        label_counts[lab] += 1

# Apply filtering
utterances = [utterances[i] for i in keep_idx]
labels     = [labels[i] for i in keep_idx]
utt_ids    = [utt_ids[i] for i in keep_idx]
user_ids   = [user_ids[i] for i in keep_idx]
topics     = [topics[i] for i in keep_idx]

print("Final counts per label:")
from collections import Counter
print(Counter(labels))

Final counts per label:
Counter({'right-leaning': 900, 'neutral': 741, 'left-leaning': 731})


In [9]:
# make a user-based split
import numpy as np
rng = np.random.default_rng(42)
uniq_users = np.array(sorted(set(user_ids)))
rng.shuffle(uniq_users)
cut = int(0.8 * len(uniq_users))
train_users = set(uniq_users[:cut])
val_users   = set(uniq_users[cut:])

train_idx = np.array([i for i,u in enumerate(user_ids) if u in train_users])
val_idx   = np.array([i for i,u in enumerate(user_ids) if u in val_users])

# Sanity
assert len(utterances) == len(labels) == len(utt_ids) == len(user_ids)

print(f"Kept {len(utterances)} labeled utterances.")

Kept 2372 labeled utterances.


In [10]:
from collections import Counter
import numpy as np

order = ['left-leaning', 'neutral', 'right-leaning']  # desired print order

# --- overall counts ---
all_counts = Counter(labels)
total = len(labels)
print("Overall:")
for k in order:
    c = all_counts.get(k, 0)
    print(f"  {k:13s}: {c:6d} ({c/total:.1%})")

# --- per-split counts (uses your train_idx / val_idx) ---
def counts_for_idx(idx):
    return Counter(labels[i] for i in idx)

print("\nTrain split:")
train_counts = counts_for_idx(train_idx)
for k in order:
    c = train_counts.get(k, 0)
    print(f"  {k:13s}: {c:6d} ({c/len(train_idx):.1%})")

print("\nVal split:")
val_counts = counts_for_idx(val_idx)
for k in order:
    c = val_counts.get(k, 0)
    print(f"  {k:13s}: {c:6d} ({c/len(val_idx):.1%})")

# --- optional: class weights aligned with STANCE2ID (for weighted loss) ---
num_classes = len(STANCE2ID)
class_weights = np.zeros(num_classes, dtype=np.float32)
for stance, idx in STANCE2ID.items():
    freq = all_counts.get(stance, 0)
    class_weights[idx] = (total / (num_classes * max(freq, 1)))  # inverse frequency

print("\nClass weights (index order per STANCE2ID):", class_weights)

Overall:
  left-leaning :    731 (30.8%)
  neutral      :    741 (31.2%)
  right-leaning:    900 (37.9%)

Train split:
  left-leaning :    583 (30.7%)
  neutral      :    590 (31.1%)
  right-leaning:    724 (38.2%)

Val split:
  left-leaning :    148 (31.2%)
  neutral      :    151 (31.8%)
  right-leaning:    176 (37.1%)

Class weights (index order per STANCE2ID): [1.0670265 1.0816233 0.8785185]


In [26]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch, numpy as np
from transformers import EarlyStoppingCallback
from torch.utils.data import DataLoader, WeightedRandomSampler  # <-- add this
from collections import Counter
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import Trainer
import torch.nn as nn
from torch.nn.functional import softmax


torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# STANCE2ID = {"neutral":0, "left-leaning":1, "right-leaning":2}
ID2STANCE = {v:k for k,v in STANCE2ID.items()}
MAX_LEN = 128

# ==== you provide ====
# utterances: list[str]; labels: list[str] aligned with utterances
# Prefer split by USER so test utterances come from unseen users.
utterances = utterances   # texts
labels     = labels
train_idx, val_idx = train_idx, val_idx # indices for your split (ideally user-based)
# =====================

tok = AutoTokenizer.from_pretrained("vinai/bertweet-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "vinai/bertweet-base", num_labels=3, id2label=ID2STANCE, label2id=STANCE2ID
)

# Probs
def get_probs(text, target):
    # target-aware formatting (recommended)
    s = f"Target: {target} </s> Post: {text}"
    batch = tok(s, return_tensors="pt", truncation=True, max_length=128)
    with torch.no_grad():
        logits = mdl(**batch).logits  # [1,3]
    return softmax(logits, dim=1).squeeze(0).tolist()  # [p_neutral, p_left, p_right]

class StanceDS(Dataset):
    def __init__(self, texts, labs):
        self.enc = tok(texts, truncation=True, padding="max_length", max_length=MAX_LEN, return_tensors="pt", pad_to_multiple_of=8)
        self.y = torch.tensor([STANCE2ID[l] for l in labs], dtype=torch.long)
    def __len__(self): return len(self.y)
    def __getitem__(self, i):
        item = {k:v[i] for k,v in self.enc.items()}
        item["labels"] = self.y[i]
        return item

train_ds = StanceDS([utterances[i] for i in train_idx], [labels[i] for i in train_idx])
val_ds   = StanceDS([utterances[i] for i in val_idx],   [labels[i] for i in val_idx])


# === soft class weights from the TRAIN SPLIT ===
train_counts = Counter(labels[i] for i in train_idx)
num_classes = len(STANCE2ID)
total_train = len(train_idx)

weights_np = np.zeros(num_classes, dtype=np.float32)
for stance, idx in STANCE2ID.items():
    freq = max(train_counts.get(stance, 1), 1)
    weights_np[idx] = total_train / (num_classes * freq)  # inverse frequency

class_weights = torch.tensor(weights_np, dtype=torch.float)
# soften the spread: mean=1.0, clamp to a narrow band
class_weights = class_weights / class_weights.mean()
class_weights = torch.clamp(class_weights, 0.9, 1.2)

LABEL_SMOOTH = 0.00  # label smoothing amount

# === Trainer with weighted CE + label smoothing (only change to loss) ===
class WeightedTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        if isinstance(class_weights, np.ndarray):
            class_weights = torch.tensor(class_weights, dtype=torch.float)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        w = None
        if self.class_weights is not None:
            w = self.class_weights.to(device=logits.device, dtype=logits.dtype)

        loss_fct = nn.CrossEntropyLoss(weight=w, label_smoothing=LABEL_SMOOTH)
        loss = loss_fct(logits, labels)

        inputs["labels"] = labels  # put back for HF internals
        return (loss, outputs) if return_outputs else loss


def compute_metrics(eval_pred):
    logits, y_true = eval_pred
    y_pred = logits.argmax(axis=-1)
    acc = accuracy_score(y_true, y_pred)
    p, r, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="macro", zero_division=0
    )
    return {
        "accuracy": acc,
        "macro_precision": p,
        "macro_recall": r,
        "macro_f1": f1,
    }

# init trainining argument
args = TrainingArguments(
    "stance-clf",
    learning_rate=3e-5,                 # start here for DeBERTa-base
    per_device_train_batch_size=16,      # L4: easy fit
    per_device_eval_batch_size=32,
    # gradient_accumulation_steps = 4,
    num_train_epochs=11,                 # allow up to 5 but early-stop
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",   # <-- change
    greater_is_better=True,             # <-- change
    warmup_ratio=0.1,
    weight_decay=0.01,
    max_grad_norm=1.0,
    fp16=True,                          # <-- put here, not in Trainer
    optim="adamw_torch",
    report_to=[],
    logging_strategy="epoch",       # <-- add this
    logging_first_step=True
)

# Init trainer
trainer = WeightedTrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tok,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    # class_weights=class_weights,   # <--- NEW
)
trainer.train()
trainer.save_model("stance-clf-best")
tok.save_pretrained("stance-clf-best")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2534376713.py:77: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,Macro Precision,Macro Recall,Macro F1
1,1.055600,0.874349,0.616842,0.656273,0.603614,0.604474
2,0.770600,0.695678,0.711579,0.712055,0.713163,0.711552
3,0.510700,0.746327,0.709474,0.708790,0.710821,0.709132
4,0.304800,0.867979,0.722105,0.728639,0.724602,0.722194
5,0.174400,1.100212,0.709474,0.731280,0.704639,0.706510
6,0.075700,1.279900,0.707368,0.712914,0.709239,0.705814
7,0.040700,1.458707,0.711579,0.715940,0.714058,0.711930


('stance-clf-best/tokenizer_config.json',
 'stance-clf-best/special_tokens_map.json',
 'stance-clf-best/vocab.txt',
 'stance-clf-best/bpe.codes',
 'stance-clf-best/added_tokens.json')

In [29]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

out = trainer.predict(val_ds)
y_pred = np.argmax(out.predictions, axis=1)
y_true = out.label_ids

print(classification_report(
    y_true, y_pred,
    target_names=["neutral","left-leaning","right-leaning"],
    digits=2
))
print("Confusion matrix (rows=true, cols=pred):")
print(confusion_matrix(y_true, y_pred))

               precision    recall  f1-score   support

      neutral       0.78      0.68      0.72       151
 left-leaning       0.66      0.81      0.73       148
right-leaning       0.75      0.69      0.72       176

     accuracy                           0.72       475
    macro avg       0.73      0.72      0.72       475
 weighted avg       0.73      0.72      0.72       475

Confusion matrix (rows=true, cols=pred):
[[102  28  21]
 [  9 120  19]
 [ 20  35 121]]


In [40]:
import numpy as np
import torch
from torch.nn.functional import softmax

# Run prediction
pred   = trainer.predict(val_ds)
logits = torch.tensor(pred.predictions)        # [N,3]
probs  = softmax(logits, dim=1).numpy()        # softmax → probabilities

# If you want hard labels too
y_hat = probs.argmax(axis=1)

# Optional derived features for your LSTM
lean_score = probs[:,2] - probs[:,1]           # right - left
conf       = 1.0 - probs[:,0]                  # not-neutral confidence

# IMPORTANT PART: get the IDs that match val_ds
val_ids = val_ds.ids   # if you followed my suggestion and stored IDs in your dataset

# Now save everything
import pandas as pd
df = pd.DataFrame({
    "utt_id":    val_ids,        # replace your_ids_array with this
    "p_neutral": probs[:,0],
    "p_left":    probs[:,1],
    "p_right":   probs[:,2],
    "lean":      lean_score,
    "conf":      conf
})
df.to_csv("stance_probs.csv", index=False)

In [33]:
# How many rows in each split?
print(len(train_df), "train rows")
print(len(dev_df), "dev rows")
if maybe_test:
    print(len(maybe_test[0]), "test rows")

# Peek at first few rows
train_df.head()

2814 train rows
100 dev rows
1249 test rows


,id,target,text,raw_stance,stance,split,opinion_towards,sentiment
0,101,Atheism,dear lord thank u for all of ur blessings forg...,against,right-leaning,train,OTHER,POSITIVE
1,102,Atheism,"Blessed are the peacemakers, for they shall be...",against,right-leaning,train,OTHER,POSITIVE
2,103,Atheism,I am not conformed to this world. I am transfo...,against,right-leaning,train,OTHER,POSITIVE
3,104,Atheism,Salah should be prayed with #focus and #unders...,against,right-leaning,train,OTHER,POSITIVE
4,105,Atheism,And stay in your houses and do not display you...,against,right-leaning,train,OTHER,NEGATIVE


In [43]:
pd.crosstab(train_df["target"], train_df["stance"])

stance,left-leaning,neutral,right-leaning
target,,,
Atheism,92,117,304
Climate Change is a Real Concern,212,168,15
Feminist Movement,210,126,328
Hillary Clinton,112,166,361
Legalization of Abortion,105,164,334
